In [1]:
from input_timeout import input_to
from imp import reload
import utils
from keras import backend as K
from keras.layers.convolutional import Conv2D
import vgg16x

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



Using gpu device 0: GeForce GTX 1070 (CNMeM is enabled with initial size: 85.0% of memory, cuDNN 5110)


Using Theano backend.


In [2]:
from keras.models import Sequential
import math

In [3]:
K.set_image_dim_ordering('th')
reload(vgg16x);reload(utils)

<module 'utils' from '/home/x991/Works/dogs_cats/utils.py'>

In [5]:
model = utils.vgg_ft(2)

(None, 3, 224, 224)


(None, 64, 112, 112)


(None, 128, 56, 56)


(None, 256, 28, 28)


(None, 512, 14, 14)


(None, 512, 7, 7)


vgg16 is initialized.


In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_14 (ZeroPaddi (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_16 (ZeroPaddi (None, 64, 114, 114)      0         
__________

conv2d_19 (Conv2D)           (None, 256, 56, 56)       590080    
_________________________________________________________________
zero_padding2d_20 (ZeroPaddi (None, 256, 58, 58)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 256, 56, 56)       590080    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 256, 28, 28)       0         
_________________________________________________________________
zero_padding2d_21 (ZeroPaddi (None, 256, 30, 30)       0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 512, 28, 28)       1180160   
_________________________________________________________________
zero_padding2d_22 (ZeroPaddi (None, 512, 30, 30)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 512, 28, 28)       2359808   
__________

conv2d_25 (Conv2D)           (None, 512, 14, 14)       2359808   
_________________________________________________________________
zero_padding2d_26 (ZeroPaddi (None, 512, 16, 16)       0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 512, 14, 14)       2359808   
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 512, 7, 7)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 4096)              102764544 
_________________________________________________________________
dropout_3 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 4096)              16781312  
__________

Total params: 134,268,738
Trainable params: 8,194
Non-trainable params: 134,260,544
_________________________________________________________________


In [7]:
model_path = './results_redux/'
model_name = 'ft7_17-08-02_22:50:23.h5'

In [8]:
model.load_weights(model_path + model_name)

In [9]:
layers = model.layers

In [10]:
for index,layer in enumerate(layers):
    print(type(layer))

<class 'keras.layers.core.Lambda'>
<class 'keras.layers.convolutional.ZeroPadding2D'>
<class 'keras.layers.convolutional.Conv2D'>
<class 'keras.layers.convolutional.ZeroPadding2D'>
<class 'keras.layers.convolutional.Conv2D'>
<class 'keras.layers.pooling.MaxPooling2D'>
<class 'keras.layers.convolutional.ZeroPadding2D'>
<class 'keras.layers.convolutional.Conv2D'>
<class 'keras.layers.convolutional.ZeroPadding2D'>
<class 'keras.layers.convolutional.Conv2D'>
<class 'keras.layers.pooling.MaxPooling2D'>
<class 'keras.layers.convolutional.ZeroPadding2D'>
<class 'keras.layers.convolutional.Conv2D'>
<class 'keras.layers.convolutional.ZeroPadding2D'>
<class 'keras.layers.convolutional.Conv2D'>
<class 'keras.layers.convolutional.ZeroPadding2D'>
<class 'keras.layers.convolutional.Conv2D'>
<class 'keras.layers.pooling.MaxPooling2D'>
<class 'keras.layers.convolutional.ZeroPadding2D'>
<class 'keras.layers.convolutional.Conv2D'>
<class 'keras.layers.convolutional.ZeroPadding2D'>
<class 'keras.layers.c

In [11]:
last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Conv2D][-1]

In [12]:
layers[last_conv_idx]

In [13]:
conv_layers = layers[:last_conv_idx+1]

In [14]:
conv_model = Sequential(conv_layers)
fc_layers = layers[last_conv_idx+1:]

In [15]:
batch_size = 64

In [16]:
batches = utils.get_batches('./train_redux', shuffle=False, batch_size=batch_size)
val_batches = utils.get_batches('./valid_redux', shuffle=False, batch_size=batch_size)

val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = utils.onehot(val_classes)
trn_labels = utils.onehot(trn_classes)

Found 23000 images belonging to 2 classes.


Found 2000 images belonging to 2 classes.


In [17]:
trn_features = conv_model.predict_generator(batches, math.ceil(batches.samples / batch_size), verbose=1)
print(trn_features.shape)

  1/360 [..............................] - ETA: 429s

  2/360 [..............................] - ETA: 389s

  3/360 [..............................] - ETA: 375s

  4/360 [..............................] - ETA: 367s

  5/360 [..............................] - ETA: 362s

...

357/360 [============================>.] - ETA: 3s

358/360 [============================>.] - ETA: 2s

359/360 [============================>.] - ETA: 1s

360/360 [==============================] - 366s   

(23000, 512, 14, 14)


In [19]:
utils.save_array('train_convlayer_features.bc', trn_features)

In [20]:
val_features = conv_model.predict_generator(val_batches, math.ceil(val_batches.samples / batch_size))
print(val_features.shape)

(2000, 512, 14, 14)


In [22]:
utils.save_array('valid_convlayer_features.bc', val_features)

In [23]:
trn_features0 = utils.load_array('train_convlayer_features.bc')
val_features0 = utils.load_array('valid_convlayer_features.bc')
print(trn_features0.shape)
print(val_features0.shape)

(23000, 512, 14, 14)
(2000, 512, 14, 14)


In [24]:
def proc_wgts(layer): return [o/2 for o in layer.get_weights()]

In [29]:
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers.pooling import MaxPooling2D

In [31]:
opt = RMSprop(lr=0.00001, rho=0.7)

In [32]:
def get_fc_model():
    model = Sequential([
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.),
        Dense(4096, activation='relu'),
        Dropout(0.),
        Dense(2, activation='softmax')
        ])

    for l1,l2 in zip(model.layers, fc_layers): l1.set_weights(proc_wgts(l2))

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [33]:
fc_model = get_fc_model()

In [34]:
fc_model.fit(trn_features, trn_labels, nb_epoch=8, 
             batch_size=batch_size, validation_data=(val_features, val_labels))

/home/x991/anaconda3/lib/python3.5/site-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Problem occurred during compilation with the command line below:
/usr/bin/g++ -shared -g -O3 -fno-math-errno -Wno-unused-label -Wno-unused-variable -Wno-write-strings -march=broadwell -mmmx -mno-3dnow -msse -msse2 -msse3 -mssse3 -mno-sse4a -mcx16 -msahf -mmovbe -maes -mno-sha -mpclmul -mpopcnt -mabm -mno-lwp -mfma -mno-fma4 -mno-xop -mbmi -mbmi2 -mno-tbm -mavx -mavx2 -msse4.2 -msse4.1 -mlzcnt -mno-rtm -mno-hle -mrdrnd -mf16c -mfsgsbase -mrdseed -mprfchw -madx -mfxsr -mxsave -mxsaveopt -mno-avx512f -mno-avx512er -mno-avx512cd -mno-avx512pf -mno-prefetchwt1 -mclflushopt -mxsavec -mxsaves -mno-avx512dq -mno-avx512bw -mno-avx512vl -mno-avx512ifma -mno-avx512vbmi -mno-clwb -mno-pcommit -mno-mwaitx --param l1-cache-size=32 --param l1-cache-line-size=64 --param l2-cache-size=6144 -mtune=generic -DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION -m64 -fPIC -I/home/x991/anaconda3/lib/python3.5/site-packages/numpy/core/include -I/home/x991/anaconda3/include/python3.5m -I/home/x991/anaconda3/lib/python3

OSError: [Errno 12] Cannot allocate memory